In [1]:
import pandas as pd
import re

def dividir_ubicacion(ubicacion_str):
    """
    Divide la cadena de ubicación en departamento, provincia y distrito
    """
    if pd.isna(ubicacion_str) or not isinstance(ubicacion_str, str):
        return {"departamento": "", "provincia": "", "distrito": ""}
    
    try:
        partes = [parte.strip() for parte in ubicacion_str.split(',')]
        
        departamento = partes[0] if len(partes) > 0 else ""
        provincia = partes[1] if len(partes) > 1 else ""
        
        distrito = ""
        if len(partes) > 2:
            distrito_str = partes[2]
            if 'distrito:' in distrito_str.lower():
                distrito_match = re.search(r'distrito:\s*(.+)', distrito_str, re.IGNORECASE)
                distrito = distrito_match.group(1).strip() if distrito_match else distrito_str
            else:
                distrito = distrito_str
        
        return {
            "departamento": departamento,
            "provincia": provincia,
            "distrito": distrito
        }
    except:
        return {"departamento": "", "provincia": "", "distrito": ""}

def procesar_censo_completo(df, año_censo):
    """
    Procesa el censo y divide la ubicación en departamento, provincia y distrito.
    Estandariza los valores de la columna 'tipo_area'.
    """
    datos = []
    ubigeo_actual = ""
    ubicacion_actual = ""
    
    for i in range(len(df)):
        fila = df.iloc[i].tolist()
        
        # Lógica para manejar los encabezados (línea de UBIGEO)
        if len(fila) > 1 and pd.notna(fila[1]) and 'AREA #' in str(fila[1]):
            match_ubigeo = re.search(r'AREA #\s*(\d{6})', str(fila[1]))
            if match_ubigeo:
                ubigeo_actual = match_ubigeo.group(1)
            
            if len(fila) > 2 and pd.notna(fila[2]):
                ubicacion_actual = str(fila[2]).strip()
            continue
        
        # Lógica para manejar las filas de datos
        if (len(fila) > 1 and pd.notna(fila[1]) and 
            any(x in str(fila[1]) for x in ['Urbano censal', 'Rural censal', 'Total'])):
            
            try:
                tipo_area_original = str(fila[1]).strip()
                poblacion_str = str(fila[2]).replace(' ', '').replace(',', '')
                poblacion = int(float(poblacion_str))
                
                ubicacion_dividida = dividir_ubicacion(ubicacion_actual)
                
                # CORRECCIÓN: Estandarizar 'tipo_area'
                if tipo_area_original == 'Urbano censal':
                    tipo_area_estandar = 'Urbano'
                elif tipo_area_original == 'Rural censal':
                    tipo_area_estandar = 'Rural'
                else:
                    tipo_area_estandar = tipo_area_original

                datos.append({
                    'ubigeo': ubigeo_actual,
                    'departamento': ubicacion_dividida['departamento'],
                    'provincia': ubicacion_dividida['provincia'],
                    'distrito': ubicacion_dividida['distrito'],
                    'tipo_area': tipo_area_estandar,
                    'poblacion': poblacion,
                    'anio': año_censo
                })
            except Exception as e:
                print(f"Error procesando fila {i}: {e}")
                continue
    
    return pd.DataFrame(datos)

print("🚀 INICIANDO PROCESAMIENTO COMPLETO")
ruta = r"D:\Mateo\ICSI\Proyecto\Data\Raw\Poblacion\Data\Inputs\Censo\censo_pob_area_2017.xlsx"
df = pd.read_excel(ruta, header=None)

# Procesar con división de ubicación
resultado = procesar_censo_completo(df, 2017)

if not resultado.empty:
    print(f"\n✅ PROCESAMIENTO COMPLETO EXITOSO!")
    print(f"📊 Total de registros: {len(resultado)}")
    
    # Mostrar estadísticas
    print(f"\n📈 ESTADÍSTICAS:")
    print(f"Departamentos únicos: {resultado['departamento'].nunique()}")
    print(f"Provincias únicas: {resultado['provincia'].nunique()}")
    print(f"Distritos únicos: {resultado['distrito'].nunique()}")
    print(f"Tipos de área: {resultado['tipo_area'].unique()}")
    
    print(f"\n👀 PRIMEROS 10 REGISTROS:")
    print(resultado[['ubigeo', 'departamento', 'provincia', 'distrito', 'tipo_area', 'poblacion']].head(10))
    
    # Guardar resultados
    ruta_destino = r"D:\Mateo\ICSI\Proyecto\Data\Raw\Poblacion\Data\Processing\pob_censo_area_2017.csv"
    resultado.to_csv(ruta_destino, index=False, encoding='latin1')
    print(f"\n💾 Archivo guardado como 'pob_censo_area_2017.csv'")
else:
    print("❌ No se encontraron datos")
    print("🔍 Mostrando estructura del archivo:")
    for i in range(15):
        fila = df.iloc[i].tolist()
        valores = [str(x) for x in fila if pd.notna(x)]
        if valores:
            print(f"Fila {i}: {valores}")

🚀 INICIANDO PROCESAMIENTO COMPLETO

✅ PROCESAMIENTO COMPLETO EXITOSO!
📊 Total de registros: 5550

📈 ESTADÍSTICAS:
Departamentos únicos: 25
Provincias únicas: 202
Distritos únicos: 1719
Tipos de área: ['Urbano' 'Rural' 'Total']

👀 PRIMEROS 10 REGISTROS:
   ubigeo departamento    provincia     distrito tipo_area  poblacion
0  010101     Amazonas  Chachapoyas  Chachapoyas    Urbano      35233
1  010101     Amazonas  Chachapoyas  Chachapoyas     Rural        634
2  010101     Amazonas  Chachapoyas  Chachapoyas     Total      35867
3  010102     Amazonas  Chachapoyas     Asunción    Urbano        155
4  010102     Amazonas  Chachapoyas     Asunción     Rural        127
5  010102     Amazonas  Chachapoyas     Asunción     Total        283
6  010103     Amazonas  Chachapoyas       Balsas    Urbano        371
7  010103     Amazonas  Chachapoyas       Balsas     Rural        861
8  010103     Amazonas  Chachapoyas       Balsas     Total       1232
9  010104     Amazonas  Chachapoyas        Chet